# Program to make files for the NA basin, land, and all buffer regions 

In [2]:
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import LineString, Point
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from tc_functions.loc_constraints import landfall_filter, remove_unwanted_points
from tc_functions.loc_constraints import get_landfall_points, get_ocean_points
from tc_functions.plotting import plot_tc_points

distance = ['100km', '200km', '300km']    #list of buffer regions
general_model = 'EC-Earth3P-HR'
model = ['EC-Earth3P-HR(19852014)(1)', 'EC-Earth3P-HR(19852014)(2)', 'EC-Earth3P-HR(20212050)(1)', 
         'EC-Earth3P-HR(20212050)(2)']    #list of models
todays_date = '2022-01-11'                #YYYY-MM-DD
                                          
for i in range(len(model)):
    
    print(f'{model[i]}:')     #open track file and extract values
        
    track_file = f'HighResMIP/{general_model}/na_basin/{model[i]}.NA.storms.nc'
    DS = xr.open_dataset(track_file)
    lons = DS.clon.values
    lats = DS.clat.values
    max_w = DS.vmax_2D.values
    try:
        time = DS.time_byte.values
    except AttributeError:
        time = DS.time_str.values
    DS.close()
    
    us_shapefile = 'shapefiles/eastern_us.shp'
    
    #find all TCs that make landfall with the eastern US coast. Save to NetCDF file
    
    print('Landfall storms')
    print(np.shape(lons))
    lons1, lats1, max_w1, time1 = landfall_filter(lons, lats, max_w, time, us_shapefile)  
        
    new_ds1 = xr.Dataset(
        data_vars = dict(
        clon = (["stormID", "time"], lons1),
        clat = (["stormID", "time"], lats1),
        vmax_2D = (["stormID", "time"], max_w1),
        time_byte = (["stormID", "time"], time1)
        ),
        attrs = dict(
        description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, " 
                    "and times for all NA landfalling storms (tropical depressions not included) " 
                    f"from the model {model[i]}.",
        author = "Justin Willson",
        creation_date = todays_date
        ),
    )

    new_ds1.to_netcdf(f"HighResMIP/{general_model}/na_basin/{model[i]}.NA.landfalling.storms.nc")
    
    #find all points on land. Save to NetCDF file
    
    print('Landfall points')
    print(np.shape(lons1))
    lons2, lats2, max_w2, time2 = get_landfall_points(lons1, lats1, max_w1, time1, us_shapefile)  
                                          
    new_ds2 = xr.Dataset(
        data_vars = dict(
        clon = (["stormID", "time"], lons2),
        clat = (["stormID", "time"], lats2),
        vmax_2D = (["stormID", "time"], max_w2),
        time_byte = (["stormID", "time"], time2)
        ),
        attrs = dict(
        description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, " 
                    "and times for all storm (tropical depressions not included) points that are on land "
                    f"for the model {model[i]}.",
        author = "Justin Willson",
        creation_date = todays_date
        ),
    )

    new_ds2.to_netcdf(f"HighResMIP/{general_model}/na_basin/{model[i]}.NA.landfalling.storms.land.pts.nc")
    
    
    for j in range(len(distance)):
        
        print(f'{distance[j]}:')
        
        buffer_shapefile = f'shapefiles/eastern_us_{distance[j]}.shp'
        
        #find all storms that intersect the specified buffer region. Save to NetCDF file
        
        print(f'Landfall filter {distance[j]}')
        print(np.shape(lons))
        lons3, lats3, max_w3, time3 = landfall_filter(lons, lats, max_w, time, buffer_shapefile) 
        
        new_ds3 = xr.Dataset(
            data_vars = dict(
            clon = (["stormID", "time"], lons3),
            clat = (["stormID", "time"], lats3),
            vmax_2D = (["stormID", "time"], max_w3),
            time_byte = (["stormID", "time"], time3)
            ),
            attrs = dict(
            description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, " 
                          "and times for all NA storms (tropical depressions not included) that come " 
                          f"within {distance[j]} of the eastern US for the model {model[i]}.",
            author = "Justin Willson",
            creation_date = todays_date
            ),
        )

        new_ds3.to_netcdf(f"HighResMIP/{general_model}/{distance[j]}/{model[i]}.NA.landfalling.storms.{distance[j]}.nc")
        
        #find all points outside of the region specified by 'eastern_us.shp'. Only TCs that intersect
        #the specified buffer region are considered. Save to NetCDF file
    
        print('Ocean points')
        print(np.shape(lons3))
        lons4, lats4, max_w4, time4 = get_ocean_points(lons3, lats3, max_w3, time3, us_shapefile) 
        
        new_ds4 = xr.Dataset(
            data_vars = dict(
            clon = (["stormID", "time"], lons4),
            clat = (["stormID", "time"], lats4),
            vmax_2D = (["stormID", "time"], max_w4),
            time_byte = (["stormID", "time"], time4)
            ),
            attrs = dict(
            description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, " 
                          "and times for all storm (tropical depressions not included) points that " 
                         f"are in the ocean from the model {model[i]}.",
            author = "Justin Willson",
            creation_date = todays_date
            ),
        )

        new_ds4.to_netcdf(f"HighResMIP/{general_model}/{distance[j]}/{model[i]}.NA."
                          f"landfalling.storms.{distance[j]}.ocean.pts.nc")
        
        #find all points in the specified buffer region. remove points in Canada,
        #the midwest, and the southwest. Save to NetCDF file
        
        print(f'{distance[j]} buffer points')
        print(np.shape(lons4))
        lons5, lats5, max_w5, time5 = get_landfall_points(lons4, lats4, max_w4, time4, buffer_shapefile)
        lons5, lats5, max_w5, time5 = remove_unwanted_points(lons5, lats5, max_w5, time5)
        
        new_ds5 = xr.Dataset(
        data_vars = dict(
            clon = (["stormID", "time"], lons5),
            clat = (["stormID", "time"], lats5),
            vmax_2D = (["stormID", "time"], max_w5),
            time_byte = (["stormID", "time"], time5)
            ),
            attrs = dict(
            description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, "
                          "and times for all landfalling storms (tropical depressions not included) "
                         f"in the eastern U.S. from the model {model[i]}. Only points where "
                         f"the storm is within {distance[j]} of land are included.",
            author = "Justin Willson",
            creation_date = todays_date
            ),
        )

        new_ds5.to_netcdf(f"HighResMIP/{general_model}/{distance[j]}/{model[i]}.NA."
                          f"landfalling.storms.{distance[j]}.buffer.pts.nc")
                          
        #plot the results for each distance as a check for accuracy
        
        gdf = gpd.read_file("shapefiles/eastern_us.shp")              #open shapefile and create geodataframe
        gdf = gdf.to_crs("EPSG:4326")                                 #convert to lat and lon from platecarree

        plt.figure(figsize=(12,7))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-63, -115, 21, 55], crs=ccrs.PlateCarree())    #plot eastern U.S.
        ax.set_title(f'{model[i]} {distance[j]}', fontsize=20)
        
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0)
        gl.xlabels_top = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
        gl.ylabel_style = {'size': 14}
        
        gdf.plot(ax=ax, color='w', edgecolor='k')                     #plot the shapefile 
        plot_tc_points(lons5, lats5)                                  #plot the buffer points
        plt.savefig(f'HighResMIP/{general_model}/{distance[j]}/A_{model[i]}_{distance[j]}.png')
        plt.close()

EC-Earth3P-HR(19852014)(1):
Landfall storms
(55, 106)
(10, 106)
Landfall points
(10, 106)
(10, 106)
100km:
Landfall filter 100km
(55, 106)
(11, 106)
Ocean points
(11, 106)
(11, 106)
100km buffer points
(11, 106)
(11, 106)
200km:
Landfall filter 200km
(55, 106)
(11, 106)
Ocean points
(11, 106)
(11, 106)
200km buffer points
(11, 106)
(11, 106)
300km:
Landfall filter 300km
(55, 106)
(11, 106)
Ocean points
(11, 106)
(11, 106)
300km buffer points
(11, 106)
(11, 106)
EC-Earth3P-HR(19852014)(2):
Landfall storms
(47, 94)
(7, 94)
Landfall points
(7, 94)
(7, 94)
100km:
Landfall filter 100km
(47, 94)
(8, 94)
Ocean points
(8, 94)
(8, 94)
100km buffer points
(8, 94)
(8, 94)
200km:
Landfall filter 200km
(47, 94)
(12, 94)
Ocean points
(12, 94)
(12, 94)
200km buffer points
(12, 94)
(12, 94)
300km:
Landfall filter 300km
(47, 94)
(14, 94)
Ocean points
(14, 94)
(14, 94)
300km buffer points
(14, 94)
(14, 94)
EC-Earth3P-HR(20212050)(1):
Landfall storms
(58, 96)
(11, 96)
Landfall points
(11, 96)
(11, 96)
10

### If the model has more than one ensemble run, combine the resulting buffer files into a larger file for each distance

In [10]:
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from tc_functions.plotting import plot_tc_points

todays_date = '2022-01-12'       #YYYY-MM-DD
general_model = 'EC-Earth3P-HR'
distance = ['100km', '200km', '300km', 'na_basin']
model = ['EC-Earth3P-HR(19852014)', 'EC-Earth3P-HR(20212050)']
                                 
individual_model = [['EC-Earth3P-HR(19852014)(1)', 'EC-Earth3P-HR(19852014)(2)'], 
                     ['EC-Earth3P-HR(20212050)(1)', 'EC-Earth3P-HR(20212050)(2)']]

for i in range(len(model)):
    for j in range(len(distance)):
        
        nstorms = []
        ntimes = []
        
        for k in range(len(individual_model[i])):
            
            if distance[j] == 'na_basin':
                track_file = f"HighResMIP/{general_model}/{distance[j]}/{individual_model[i][k]}.NA.storms.nc"
            else:
                track_file = f"HighResMIP/{general_model}/{distance[j]}/{individual_model[i][k]}.NA.landfalling.storms.{distance[j]}.buffer.pts.nc"
            DS = xr.open_dataset(track_file)
            ds_lons = DS.clon.values
            DS.close()
    
            nstorms.append(np.shape(ds_lons)[0])
            ntimes.append(np.shape(ds_lons)[1])
    
        comb_lons = np.full((sum(nstorms), max(ntimes)), 0, dtype = np.float32)   # initialize combined arrays 
        comb_lats = np.full((sum(nstorms), max(ntimes)), 0, dtype = np.float32)
        comb_maxw = np.full((sum(nstorms), max(ntimes)), 0, dtype = np.float32)
        comb_time = np.full((sum(nstorms), max(ntimes)), b'', dtype = '|S19')
        
        num = 0
        for k in range(len(individual_model[i])):
            
            if distance[j] == 'na_basin':
                track_file = f"HighResMIP/{general_model}/{distance[j]}/{individual_model[i][k]}.NA.storms.nc"
            else:
                track_file = f"HighResMIP/{general_model}/{distance[j]}/{individual_model[i][k]}.NA.landfalling.storms.{distance[j]}.buffer.pts.nc"
            DS = xr.open_dataset(track_file)
            ds_lons = DS.clon.values
            ds_lats = DS.clat.values
            ds_maxw = DS.vmax_2D.values
            ds_time = DS.time_byte.values
            DS.close()
            
            for n in range(nstorms[k]):
                for m in range(ntimes[k]):
                    comb_lons[num,m] = ds_lons[n,m]
                    comb_lats[num,m] = ds_lats[n,m]
                    comb_maxw[num,m] = ds_maxw[n,m] 
                    comb_time[num,m] = ds_time[n,m]
                num += 1
                
        new_ds = xr.Dataset(                                  # create and save a new combined netcdf file
            data_vars = dict(
            clon = (["stormID", "time"], comb_lons),
            clat = (["stormID", "time"], comb_lats),
            vmax_2D = (["stormID", "time"], comb_maxw),
            time_byte = (["stormID", "time"], comb_time)
            ),
            attrs = dict(
            description = f"NetCDF file that combines the buffer points from the {len(individual_model[i])} "
                          f"{model[i]} ensembles.",
            author = "Justin Willson",
            creation_date = todays_date
            ),
        )

        print(new_ds)
        
        if distance[j] == 'na_basin':
            new_ds.to_netcdf(f'HighResMIP/{general_model}/{distance[j]}/{model[i]}.NA.storms.nc')
        else:
            new_ds.to_netcdf(f'HighResMIP/{general_model}/{distance[j]}/{model[i]}.NA.landfalling.storms.{distance[j]}.buffer.pts.nc')
            
        #plot the results for each distance as a check for accuracy
        
        gdf = gpd.read_file("shapefiles/eastern_us.shp")              #open shapefile and create geodataframe
        gdf = gdf.to_crs("EPSG:4326")                                 #convert to lat and lon from platecarree

        plt.figure(figsize=(12,7))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-63, -115, 21, 55], crs=ccrs.PlateCarree())    #plot eastern U.S.
        ax.set_title(f'{model[i]} {distance[j]}', fontsize=20)
        
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0)
        gl.xlabels_top = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
        gl.ylabel_style = {'size': 14}
        
        gdf.plot(ax=ax, color='w', edgecolor='k')                     #plot the shapefile 
        plot_tc_points(comb_lons, comb_lats)                          #plot the buffer points
        plt.savefig(f'HighResMIP/{general_model}/{distance[j]}/A_{model[i]}_{distance[j]}.png')
        plt.close()

<xarray.Dataset>
Dimensions:    (stormID: 19, time: 106)
Dimensions without coordinates: stormID, time
Data variables:
    clon       (stormID, time) float32 -77.51953 -77.16797 -76.81641 ... 0.0 0.0
    clat       (stormID, time) float32 33.19023 33.89267 34.24389 ... 0.0 0.0
    vmax_2D    (stormID, time) float32 12.82743 11.76152 13.25121 ... 0.0 0.0
    time_byte  (stormID, time) |S19 b'1985-09-15T18:00:00' ... b''
Attributes:
    description:    NetCDF file that combines the buffer points from the 2 EC...
    author:         Justin Willson
    creation_date:  2022-01-12
<xarray.Dataset>
Dimensions:    (stormID: 23, time: 106)
Dimensions without coordinates: stormID, time
Data variables:
    clon       (stormID, time) float32 -77.51953 -77.51953 -77.87109 ... 0.0 0.0
    clat       (stormID, time) float32 32.13657 32.48779 32.48779 ... 0.0 0.0
    vmax_2D    (stormID, time) float32 12.86843 12.73715 12.34798 ... 0.0 0.0
    time_byte  (stormID, time) |S19 b'1985-09-14T18:00:00' ...